In [ ]:
import re
import json
import pickle
import pandas as pd

In [ ]:
def json_to_df(raw):
    df = pd.DataFrame(columns=["Topic", "Question", "Answer", "Answer start", "Plausible Answer",
                                  "Plausible answer start", "Is impossible", "Context"])
    contexts = []
    row = 0
    for i in range(len(raw["data"])):
        for j in range(len(raw["data"][i]["paragraphs"])):
            contexts.append(raw["data"][i]["paragraphs"][j]['context'])
            for k in range(len(raw["data"][i]["paragraphs"][j]["qas"])):
                qa = raw["data"][i]["paragraphs"][j]["qas"][k]
                title = raw["data"][i]["title"]
                question = qa["question"]

                if(len(qa["answers"])):
                    answer = qa["answers"][0]["text"]
                    answer_start = qa["answers"][0]["answer_start"]
                else:
                    answer = None
                    answer_start = None

                if('plausible_answers' in qa.keys() and len(qa["plausible_answers"])):
                    plausible = qa["plausible_answers"][0]["text"]
                    plausible_start = qa["plausible_answers"][0]["answer_start"]
                else:
                    plausible = None
                    plausible_start = None

                is_impossible = qa["is_impossible"]
                df.loc[row] = [title, question, answer, answer_start, plausible, plausible_start, is_impossible, len(contexts)-1]
                row += 1
    return df, contexts

In [ ]:
train, train_contexts = json_to_df(json.load(open("data/train.json")))
test, test_contexts = json_to_df(json.load(open("data/validation.json")))

In [ ]:
train.to_csv("data/train.csv", index=False)
test.to_csv("data/validation.csv", index=False)

In [ ]:
split = lambda x: re.findall(r"[\w']+|[.,!?;]", x.lower()) if x is not None else None

def json_to_dct(raw):
    qas = []
    for i in range(len(raw["data"])):
        for j in range(len(raw["data"][i]["paragraphs"])):
            for k in range(len(raw["data"][i]["paragraphs"][j]["qas"])):
                qa = raw["data"][i]["paragraphs"][j]["qas"][k]
                question = qa["question"]
                if(qa["is_impossible"]):
                    answer = None
                else:
                    answer = qa["answers"][0]["text"]
                    
                is_impossible = qa["is_impossible"]
                qas.append({"question": split(question), 
                            "answer": split(answer), 
                            "context": split(raw["data"][i]["paragraphs"][j]['context'])})
    return qas

In [ ]:
train = json_to_dct(json.load(open("data/train.json")))
validation = json_to_dct(json.load(open("data/validation.json")))

In [ ]:
pickle.dump(train, open("data/train.pkl", "wb"))
pickle.dump(validation, open("data/validation.pkl", "wb"))